In [14]:
import os
import sys
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder

import os
import tarfile

import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import psutil



In [15]:

sys.path.append(os.path.abspath('..'))

from dataframe_generator import *
from images_standardizing import *

print("GPU:")
!nvidia-smi

GPU:
Wed Jul 17 16:57:19 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.02              Driver Version: 555.42.02      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650        Off |   00000000:01:00.0 Off |                  N/A |
| N/A   44C    P8              3W /   50W |    3623MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+

In [16]:
def print_memory_usage():
    print(f"Memory usage: {psutil.virtual_memory().percent}%")
    if torch.cuda.is_available():
        print(f"CUDA memory allocated: {torch.cuda.memory_allocated() / (1024 * 1024)} MB")
        print(f"CUDA memory cached: {torch.cuda.memory_reserved() / (1024 * 1024)} MB")

In [17]:

def extract_tgz(tgz_path, extract_to):
    if not os.path.exists(extract_to):
        os.makedirs(extract_to)
    
    with tarfile.open(tgz_path, 'r:gz') as tar:
        tar.extractall(path=extract_to)
        print(f"Arquivos extraídos para {extract_to}")

tgz_path = '../CBIR_15-Scene.tgz'
extract_to = '../data/'

extract_tgz(tgz_path, extract_to)

Arquivos extraídos para ../data/


In [18]:
df = create_dataframe()
df

                             image_path category
0        ../data/CBIR_15-Scene/00/1.jpg  Bedroom
1        ../data/CBIR_15-Scene/00/2.jpg  Bedroom
2        ../data/CBIR_15-Scene/00/3.jpg  Bedroom
3        ../data/CBIR_15-Scene/00/4.jpg  Bedroom
4        ../data/CBIR_15-Scene/00/5.jpg  Bedroom
...                                 ...      ...
4480  ../data/CBIR_15-Scene/14/4481.jpg    Store
4481  ../data/CBIR_15-Scene/14/4482.jpg    Store
4482  ../data/CBIR_15-Scene/14/4483.jpg    Store
4483  ../data/CBIR_15-Scene/14/4484.jpg    Store
4484  ../data/CBIR_15-Scene/14/4485.jpg    Store

[4485 rows x 2 columns]


,image_path,category
0,../data/CBIR_15-Scene/00/1.jpg,Bedroom
1,../data/CBIR_15-Scene/00/2.jpg,Bedroom
2,../data/CBIR_15-Scene/00/3.jpg,Bedroom
3,../data/CBIR_15-Scene/00/4.jpg,Bedroom
4,../data/CBIR_15-Scene/00/5.jpg,Bedroom
...,...,...
4480,../data/CBIR_15-Scene/14/4481.jpg,Store
4481,../data/CBIR_15-Scene/14/4482.jpg,Store
4482,../data/CBIR_15-Scene/14/4483.jpg,Store
4483,../data/CBIR_15-Scene/14/4484.jpg,Store


In [19]:
categories = df['category'].unique()
category_to_idx = {category: idx for idx, category in enumerate(categories)}
df['label'] = df['category'].map(category_to_idx)

encoder = OneHotEncoder(sparse_output=False)
one_hot_labels = encoder.fit_transform(df[['label']])
df = df.join(pd.DataFrame(one_hot_labels, columns=categories))

df

,image_path,category,label,Bedroom,Suburb,Industry,Kitchen,LivingRoom,Coast,Forest,Highway,InsideCity,Mountain,OpenCountry,Street,Building,Office,Store
0,../data/CBIR_15-Scene/00/1.jpg,Bedroom,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,../data/CBIR_15-Scene/00/2.jpg,Bedroom,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,../data/CBIR_15-Scene/00/3.jpg,Bedroom,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,../data/CBIR_15-Scene/00/4.jpg,Bedroom,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,../data/CBIR_15-Scene/00/5.jpg,Bedroom,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4480,../data/CBIR_15-Scene/14/4481.jpg,Store,14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4481,../data/CBIR_15-Scene/14/4482.jpg,Store,14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4482,../data/CBIR_15-Scene/14/4483.jpg,Store,14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4483,../data/CBIR_15-Scene/14/4484.jpg,Store,14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [20]:
from pytorch_ood.detector import (
    ODIN,
    EnergyBased,
    Entropy,
    KLMatching,
    Mahalanobis,
    MaxLogit,
    MaxSoftmax,
    ViM,
    RMD,
    DICE,
    SHE,
)

from pytorch_ood.model import WideResNet
from pytorch_ood.utils import OODMetrics, ToUnknown, fix_random_seed

fix_random_seed(42)

In [21]:
class CBIR15Dataset(Dataset):
    """CBIR dataset."""

    def __init__(self, df, root_dir, transform=None):
       
        self.dataframe = df
        self.df_labels = df['label'].unique()
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        image_path = self.dataframe.iloc[idx, 0]
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)

        if self.transform:
            image = self.transform(image)

        label = self.df_labels
        return image, torch.tensor(label)

In [22]:
trans = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((244, 244)),
    transforms.ToTensor(),
])

In [23]:
custom_dataset = CBIR15Dataset(df=df, root_dir='data', transform=trans)
custom_loader = DataLoader(custom_dataset, batch_size=32, shuffle=False)

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = WideResNet(num_classes=len(categories))

In [25]:
try:
    model = model.to(device).eval()
except RuntimeError as e:
    device = torch.device("cpu")
    model = model.to(device).eval()

In [27]:
def extract_features(loader, model, device):
    model.eval()
    features = []
    labels = []
    with torch.no_grad():
        for images, lbls in loader:
            images = images.to(device)
            # Extrair características em partes
            feats = model.conv1(images)
            feats = model.block1(feats)
            feats = model.block2(feats)
            feats = model.block3(feats)
            feats = model.bn1(feats)
            feats = torch.relu(feats)
            feats = torch.nn.functional.adaptive_avg_pool2d(feats, (1, 1))
            feats = feats.view(feats.size(0), -1)
            features.append(feats.cpu())
            labels.append(lbls)
            # Liberar memória
            del images, feats
            torch.cuda.empty_cache()
    features = torch.cat(features, dim=0)
    labels = torch.cat(labels, dim=0)
    return features, labels

In [28]:
try:
    in_features, in_labels = extract_features(custom_loader, model, device)
except RuntimeError as e:
    if 'out of memory' in str(e):
        print("Out of memory error caught. Clearing cache and retrying...")
        torch.cuda.empty_cache()
        in_features, in_labels = extract_features(custom_loader, model, device)
    else:
        raise e

Out of memory error caught. Clearing cache and retrying...


OutOfMemoryError: CUDA out of memory. Tried to allocate 118.00 MiB. GPU 

In [ ]:
detectors = {
	"ODIN": ODIN(model),
    "EnergyBased": EnergyBased(model),
    "Entropy": Entropy(model),
    "KLMatching": KLMatching(model),
    "Mahalanobis": Mahalanobis(model),
    "MaxLogit": MaxLogit(model),
    "MaxSoftmax": MaxSoftmax(model),
    #"ViM": ViM(model),
    "RMD": RMD(model),
    #"DICE": DICE(model),
    #"SHE": SHE(model)
}